In [143]:
from itertools import combinations

import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import random
import math

from icecream import ic


Cost: $d + (d \cdot \alpha \cdot w)^\beta$ with $\alpha \ge 0$ and $\beta \ge 0$

In [144]:
class Problem:
    _graph: nx.Graph
    _alpha: float
    _beta: float

    def __init__(
        self,
        num_cities: int,
        *,
        alpha: float = 1.0,
        beta: float = 1.0,
        density: float = 0.5,
        seed: int = 42,
    ):
        rng = np.random.default_rng(seed)
        self._alpha = alpha
        self._beta = beta
        cities = rng.random(size=(num_cities, 2))
        cities[0, 0] = cities[0, 1] = 0.5

        self._graph = nx.Graph()
        self._graph.add_node(0, pos=(cities[0, 0], cities[0, 1]), gold=0)
        for c in range(1, num_cities):
            self._graph.add_node(c, pos=(cities[c, 0], cities[c, 1]), gold=(1 + 999 * rng.random()))

        tmp = cities[:, np.newaxis, :] - cities[np.newaxis, :, :]
        d = np.sqrt(np.sum(np.square(tmp), axis=-1))
        for c1, c2 in combinations(range(num_cities), 2):
            if rng.random() < density or c2 == c1 + 1:
                self._graph.add_edge(c1, c2, dist=d[c1, c2])

        assert nx.is_connected(self._graph)

    @property
    def graph(self) -> nx.Graph:
        return nx.Graph(self._graph)

    @property
    def alpha(self):
        return self._alpha

    @property
    def beta(self):
        return self._beta

    def cost(self, path, weight):
        dist = nx.path_weight(self._graph, path, weight='dist')
        return dist + (self._alpha * dist * weight) ** self._beta

    def baseline(self):
        total_cost = 0
        for dest, path in nx.single_source_dijkstra_path(
            self._graph, source=0, weight='dist'
        ).items():
            cost = 0
            for c1, c2 in zip(path, path[1:]):
                cost += self.cost([c1, c2], 0)
                cost += self.cost([c1, c2], self._graph.nodes[dest]['gold'])
            logging.debug(
                f"dummy_solution: go to {dest} ({' > '.join(str(n) for n in path)} ({cost})"
            )
            total_cost += cost
        return total_cost

    def plot(self):
        plt.figure(figsize=(10, 10))
        pos = nx.get_node_attributes(self._graph, 'pos')
        size = [100] + [self._graph.nodes[n]['gold'] for n in range(1, len(self._graph))]
        color = ['red'] + ['lightblue'] * (len(self._graph) - 1)
        return nx.draw(self._graph, pos, with_labels=True, node_color=color, node_size=size)

In [ ]:
def precompute_distances(problem):
    """
    Calcola matrice delle distanze tra tutte le coppie di nodi nel grafo del problema.
    Restituisce una matrice 2D numpy NxN dove l'elemento (i, j) rappresenta la distanza tra il nodo i e il nodo j.
    """
    graph = problem._graph
    n = len(graph.nodes)
    #print("Pre-calcolo delle distanze in corso...")
    # Inizializza una matrice NxN con tutti zeri
    dist_matrix = np.zeros((n, n))

    all_pairs = nx.all_pairs_dijkstra_path_length(graph, weight='dist')
    # Popola la matrice delle distanze
    for src, targets in all_pairs:
        for tgt, d in targets.items():
            dist_matrix[src][tgt] = d
    #print("Pre-calcolo delle distanze completato.")
    return dist_matrix


In [146]:
# ---- FUNZIONI DI SUPPORTO ----
def getCost(problem, solution, dist_matrix):
    """Calcola il costo (Energia) della soluzione."""
    cost = 0
    weight = 0
    current = 0
    alpha = problem._alpha
    beta = problem._beta

    for node in solution:
        if node == 0:
            if current != 0:
                d = dist_matrix[current][0]
                cost += d + (alpha * d * weight) ** beta
                weight = 0
                current = 0
        else:
            d = dist_matrix[current][node]
            cost += d + (alpha * d * weight) ** beta
            weight += problem._graph.nodes[node]['gold']
            current = node

    if current != 0:
        d = dist_matrix[current][0]
        cost += d + (alpha * d * weight) ** beta

    return cost


In [147]:
def two_opt_segment(path, i, j):
    new_path = list(path)
    new_path[i:j] = reversed(new_path[i:j])
    return new_path


def insert_return(path):
    new = list(path)
    pos = random.randint(1, len(new))
    if new[pos - 1] != 0:
        new.insert(pos, 0)
    return new


def remove_return(path):
    new = list(path)
    zeros = [i for i, v in enumerate(new) if v == 0]
    if zeros:
        new.pop(random.choice(zeros))
    return new


# Thermodynamic Greedy Search + SA

In [ ]:
class ThermodynamicGreedySearch:
    def __init__(self, problem, dist_matrix,
                 initial_temp=2000.0,
                 cooling_rate=0.995,
                 min_temp=0.1,
                 iter_per_temp=50):

        self.problem = problem
        self.dist_matrix = dist_matrix
        self.n_cities = len(problem._graph.nodes)

        # parametri fisici
        self.T = initial_temp
        self.cooling_rate = cooling_rate
        self.min_temp = min_temp
        self.iter_per_temp = iter_per_temp

    def run(self):
        #print("--- Inizio Thermodynamic Greedy Search ---")

        # 1. GENERAZIONE SOLUZIONE INIZIALE (La Regola Aurea)
        # Costruiamo una soluzione greedy intelligente basata sul "Peso Futuro"
        current_solution = self._generate_golden_greedy()
        current_energy = getCost(self.problem, current_solution, self.dist_matrix)

        best_solution = list(current_solution)
        best_energy = current_energy
        #print(f"Energia Iniziale (Greedy): {best_energy:.2f}")
        
        # 2. CICLO PRINCIPALE DEL SIMULATED ANNEALING
        iteration = 0
        while self.T > self.min_temp:
            for _ in range(self.iter_per_temp):
                # a. perturbazione casuale della soluzione corrente
                move = random.choice(["2opt", "add0", "rem0"])

                if move == "2opt" and len(current_solution) >= 2:
                    i, j = sorted(random.sample(range(len(current_solution)), 2))
                    candidate = two_opt_segment(current_solution, i, j)
                elif move == "add0":
                    candidate = insert_return(current_solution)
                else:
                    candidate = remove_return(current_solution)
                
                # b. CALCOLO ENERGIA CANDIDATO
                candidate_energy = getCost(
                    self.problem, candidate, self.dist_matrix
                )

                delta = candidate_energy - current_energy

                # c. Legge di Boltzmann
                # Accettiamo se migliora (delta < 0) OPPURE se il dado di Boltzmann dice sì
                if delta < 0 or random.random() < math.exp(-delta / self.T):
                    current_solution = candidate
                    current_energy = candidate_energy
                    # Aggiorniamo il record assoluto
                    if current_energy < best_energy:
                        best_energy = current_energy
                        best_solution = list(current_solution)

            self.T *= self.cooling_rate
            iteration += 1
            if iteration % 50 == 0:
                print(f"Temp: {self.T:.2f} | Energia: {current_energy:.2f} | Record: {best_energy:.2f}")

        return best_solution, best_energy

    def _generate_initial_solution(self):
        nodes = list(range(1, self.n_cities))
        random.shuffle(nodes)
        return nodes

    def _generate_golden_greedy(self):
        """
        Genera la soluzione iniziale usando la 'Regola Aurea':
        Preferisci città vicine MA penalizza quelle pesanti se sei lontano da casa.
        """
        unvisited = set(range(1, self.n_cities))
        curr = 0
        path = []
        
        # Estraiamo i pesi (oro)
        gold = {i: self.problem._graph.nodes[i].get('gold', 0) for i in range(self.n_cities)}
        
        while unvisited:
            best_next = -1
            best_score = float('inf')
            
            for city in unvisited:
                dist = self.dist_matrix[curr][city]
                weight = gold[city]
                dist_to_home = self.dist_matrix[city][0]
                
                # FORMULA AUREA:
                # Score = Distanza * (1 + FattorePeso)
                # Il FattorePeso cresce se la città è pesante E lontana da casa
                weight_penalty = (weight * dist_to_home) / 1000.0 # Normalizzazione euristica
                
                score = dist + weight_penalty
                
                if score < best_score:
                    best_score = score
                    best_next = city
            
            path.append(best_next)
            unvisited.remove(best_next)
            curr = best_next
            
        return path

problem_instance = Problem(100, density=0.2, alpha=1, beta=1)
dist_matrix = precompute_distances(problem_instance)
problem_instance.plot()

solver = ThermodynamicGreedySearch(
    problem_instance,
    dist_matrix,
    initial_temp=20000.0,
    cooling_rate=0.995,
    min_temp=0.1,
    iter_per_temp=50
)

sol, cost = solver.run()
improvement = 100 * (problem_instance.baseline() - cost) / problem_instance.baseline()

print(f"\nCosto finale: {cost:.2f}")
print(f"Baseline: {problem_instance.baseline():.2f}")
print(f"Miglioramento: {improvement:.2f}%")


# ALGORITMI GENETICI (GA)

In [149]:
def getFastSmartCost(problem: Problem, solution: list[int], dist_matrix) -> float:
    """
    For each step, calculate the cost considering the accumulated weight using the precomputed distance matrix.
    """
    cost = 0
    weight = 0
    current_node = 0  # start from the node 0

    alpha = problem._alpha
    beta = problem._beta

    # Visit each city in the solution path
    for next_node in solution:
        # get distance from precomputed matrix
        dist_direct = dist_matrix[current_node][next_node]
        dist_to_0 = dist_matrix[current_node][0]
        dist_from_0 = dist_matrix[0][next_node]
        
        # Aggiorniamo il costo 
        cost_direct = dist_direct + (alpha * dist_direct * weight) ** beta

        cost_via_0 = dist_to_0 + (alpha * dist_to_0 * weight) ** beta + dist_from_0
        if cost_via_0 < cost_direct:
            cost += cost_via_0
            weight = 0
        else:
            cost += cost_direct
        
        weight += problem._graph.nodes[next_node].get('gold',1)
        current_node = next_node

    dist_final = dist_matrix[current_node][0]
    cost += dist_final + (alpha * dist_final * weight) ** beta

    return cost

In [150]:
def tweak(solution: list[int]) -> list[int]:
    """
    Applia un piccolo cambiamento (tweak) alla soluzione esistente
    scambiando due città nella lista.
    """
    if len(solution) < 2:
        return solution.copy()
    
    new_solution = solution.copy()
    i, j = random.sample(range(len(solution)), 2)
    new_solution[i], new_solution[j] = new_solution[j], new_solution[i]
    return new_solution

population

In [151]:
POPULATION_SIZE = 100

def init_population(problem: Problem, population_size: int) -> list[list[int]]:
    """
    Inizializza una popolazione di soluzioni casuali.
    Ogni soluzione è una permutazione casuale delle città (escluso il deposito 0).
    """
    num_cities = len(problem._graph.nodes)
    cities = list(range(1, num_cities))  # Escludo il deposito 0

    population = []
    for _ in range(population_size):
        individual = cities.copy()
        random.shuffle(individual)
        population.append(individual)
    return population

Parent Selection: Tournament Selection

In [152]:
def tournament_selection(problem: Problem, population: list[list[int]], distance_matrix, tournament_size: int = 3):
    # Seleziono k individui casuali dalla popolazione
    candidates = random.sample(population, tournament_size)

    # Valuto chi tra loro a il costo migliore (minore)
    best_candidate = None
    best_cost = float('inf')

    for cand in candidates:
        cost = getFastSmartCost(problem, cand, distance_matrix)
        if cost < best_cost:
            best_cost = cost
            best_candidate = cand
    return best_candidate

## Crossover - Order Crossover (OX)

OX necessario al fine di non avere città duplicate negli individui figli

In [153]:
def crossover(parent1: list[int], parent2: list[int]) -> list[int]:
    """
    Applica l'operatore di crossover Order Crossover (OX) per generare un figlio
    da due genitori.
    """
    size = len(parent1)

    # Scelgo due punti di taglio casuali
    start, end = sorted(random.sample(range(size), 2))

    # Creo il figlio con None e copiamo la parte centrale da parent1
    child = [None] * size
    child[start:end+1] = parent1[start:end+1] # serve "+1" per includere end

    # Riempio il resto del figlio con gli elementi di parent2 nell'ordine in cui appaiono
    # Lo faccio scorrendo il genitore2 e prendendo solo gli elementi non ancora presenti nel figlio
    p2_genes = [gene for gene in parent2 if gene not in child[start:end+1]]

    current_p2_index = 0
    for i in range(size):
        if child[i] is None:
            child[i] = p2_genes[current_p2_index]
            current_p2_index += 1
    return child

## Mutation

In [154]:
MUTATION_RATE = 0.2

def mutation(individual: list[int], mutation_rate: float=0.1) -> list[int]:
    """
    Applica la mutazione alla soluzione scambiando due città con una certa probabilità.
    """
    if random.random() < mutation_rate:
        return tweak(individual)
    else:
        return individual

## Genetic Algorithm

In [155]:
NUM_GENERATIONS = 2_000

In [ ]:
def genetic_algorithm(problem: Problem, distance_matrix, population_size: int = 50, generations: int = 1000, mutation_rate: float= 0.1):
    # Inizializzo la popolazione
    population = init_population(problem, population_size)

    best_solution = None
    best_cost = float('inf')

    #print(f"Inizio Genetic Algorithm con population_size={population_size}, generations={generations}, mutation_rate={mutation_rate}")

    for gen in range(generations):
        new_population = []

        # --- ELITISMO ---
        # Salviamo subito il migliore della vecchia generazione nella nuova popolazione
        # Questo garantisce che la qualità non peggiori
        population.sort(key=lambda x: getFastSmartCost(problem, x, distance_matrix))
        best_of_gen = population[0]
        cost_of_gen = getFastSmartCost(problem, best_of_gen, distance_matrix)

        if cost_of_gen < best_cost:
            best_cost = cost_of_gen
            best_solution = best_of_gen.copy()
            #print(f"Generation {gen}: Nuovo Record {best_cost:.2f}")

        new_population.append(best_of_gen)
        # -----------------

        # Riempiamo il resto della nuova popolazione (population_size - 1)
        while len(new_population) < population_size:
            # Parent Selection
            parent1 = tournament_selection(problem, population, distance_matrix)
            parent2 = tournament_selection(problem, population, distance_matrix)

            # Crossover
            child = crossover(parent1, parent2)

            # Mutation
            child = mutation(child, mutation_rate)

            new_population.append(child)
    
        # Sostituiamo la vecchia popolazione con la nuova
        population = new_population

    return best_solution, best_cost


In [157]:
POPULATION_SIZE = 50
MUTATION_RATE = 0.2
NUM_GENERATIONS = 2_000

# MAIN
problem_instance = Problem(100, density=0.2, alpha=1, beta=1)

# Pre-calcolo delle distanze
dist_matrix = precompute_distances(problem_instance)

sol, cost = genetic_algorithm(problem_instance, distance_matrix=dist_matrix, population_size=POPULATION_SIZE, generations=NUM_GENERATIONS, mutation_rate=MUTATION_RATE)

print(f"\n---RISULTATO FINALE---")
print(f"Costo Migliore: {cost:.2f}")
print(f"Baseline (Dijkstra): {problem_instance.baseline():.2f}")

# Controllo se ho battuto la baseline
if cost < problem_instance.baseline():
    print("VITTORIA: GA con distance matrix ha battuto la baseline!")
else:
    print("SCONFITTA: GA con distance matrix non è bastato (prova più iterazioni).")

In [ ]:
import pandas as pd
import time

def run_comparison():
   # 1. Configurazione dei Test
    test_cases = [
    # (N, Density, Alpha, Beta)
    # Casi 100 Città
    (100, 0.2, 1, 1),
    (100, 0.2, 2, 1),
    (100, 0.2, 1, 2), # Attenzione: Beta=2 fa esplodere i costi!
    (100, 1.0, 1, 1),
    (100, 1.0, 2, 1),
    (100, 1.0, 1, 2),
    
    # Casi 1000 Città (Attenzione ai tempi!)
    # Per il test rapido, magari commenta questi e provali in un secondo momento
    # (1000, 0.2, 1, 1), 
    # (1000, 1.0, 1, 1),
]
    results = []
    print(f"{'SCENARIO':<30} | {'BASELINE':<12} | {'GA':<12} | {'SA':<12} | {'WINNER':<10}")
    print("-" * 90)
  

    for n_cities, dens, alpha, beta in test_cases:
        # a. SETUP PROBLEMA
        problem_instance = Problem(n_cities, density=dens, alpha=alpha, beta=beta)
                for beta in BETAS:

                    experiment_id += 1
                    print("\n" + "=" * 80)
                    print(f"Esperimento {experiment_id}")
                    print(f"n={n}, density={density}, alpha={alpha}, beta={beta}")
                    print("=" * 80)

                    # =========================
                    # CREAZIONE PROBLEMA
                    # =========================
                    problem_instance = Problem(
                        n,
                        density=density,
                        alpha=alpha,
                        beta=beta
                    )

                    # =========================
                    # BASELINE
                    # =========================
                    start = time.time()
                    baseline_cost = problem_instance.baseline()
                    baseline_time = time.time() - start

                    print(f"Baseline cost: {baseline_cost:.2f} ({baseline_time:.2f}s)")

                    # =========================
                    # PRECOMPUTE DISTANCES
                    # =========================
                    dist_matrix = precompute_distances(problem_instance)

                    # =========================
                    # ALGORITMO 1: GENETIC
                    # =========================
                    start = time.time()
                    sol_ga, cost_ga = genetic_algorithm(
                        problem_instance,
                        distance_matrix=dist_matrix,
                        population_size=POPULATION_SIZE,
                        generations=NUM_GENERATIONS,
                        mutation_rate=MUTATION_RATE
                    )
                    ga_time = time.time() - start

                    ga_improvement = 100 * (baseline_cost - cost_ga) / baseline_cost

                    print(
                        f"GA cost: {cost_ga:.2f} "
                        f"| Improvement: {ga_improvement:.2f}% "
                        f"| Time: {ga_time:.2f}s"
                    )

                    # =========================
                    # ALGORITMO 2: SA
                    # =========================
                    solver = ThermodynamicGreedySearch(
                        problem=problem_instance,
                        dist_matrix=dist_matrix,
                        initial_temp=2000.0,
                        cooling_rate=0.995,
                        min_temp=0.1,
                        iter_per_temp=50
                    )



                    start = time.time()
                    sol_sa, cost_sa = solver.run()
                    sa_time = time.time() - start

                    sa_improvement = 100 * (baseline_cost - cost_sa) / baseline_cost

                    print(
                        f"SA cost: {cost_sa:.2f} "
                        f"| Improvement: {sa_improvement:.2f}% "
                        f"| Time: {sa_time:.2f}s"
                    )

                    # =========================
                    # SALVATAGGIO RISULTATI
                    # =========================
                    results.append({
                        "n": n,
                        "density": density,
                        "alpha": alpha,
                        "beta": beta,
                        "baseline_cost": baseline_cost,
                        "baseline_time": baseline_time,
                        "ga_cost": cost_ga,
                        "ga_time": ga_time,
                        "ga_improvement_%": ga_improvement,
                        "sa_cost": cost_sa,
                        "sa_time": sa_time,
                        "sa_improvement_%": sa_improvement,
                    })

    # =========================
    # TABELLA FINALE
    # =========================
    df = pd.DataFrame(results)

    print("\n" + "=" * 80)
    print("RISULTATI FINALI")
    print("=" * 80)
    print(df)

    return df


In [159]:
df_results = run_comparison()



Esperimento 1
n=50, density=0.2, alpha=0.5, beta=1.0
Baseline cost: 6475.52 (0.00s)
Pre-calcolo delle distanze in corso...
Pre-calcolo delle distanze completato.
Inizio Genetic Algorithm con population_size=50, generations=200, mutation_rate=0.1
Generation 0: Nuovo Record 7070.02
Generation 1: Nuovo Record 6859.74
Generation 3: Nuovo Record 6812.73
Generation 4: Nuovo Record 6780.28
Generation 5: Nuovo Record 6727.99
Generation 6: Nuovo Record 6594.03
Generation 8: Nuovo Record 6534.47
Generation 11: Nuovo Record 6531.81
Generation 12: Nuovo Record 6523.83
Generation 13: Nuovo Record 6505.96
Generation 14: Nuovo Record 6495.43
Generation 15: Nuovo Record 6474.15
Generation 19: Nuovo Record 6474.11
Generation 22: Nuovo Record 6473.45
Generation 25: Nuovo Record 6473.33
Generation 28: Nuovo Record 6473.33
Generation 31: Nuovo Record 6473.16
Generation 34: Nuovo Record 6472.62
Generation 36: Nuovo Record 6472.23
Generation 50: Nuovo Record 6472.23
Generation 60: Nuovo Record 6471.66
Gene